In [1]:
def refresh_repo():
    %cd /kaggle/working
    %rm -rf hotflip
    !git clone https://github.com/jefri021/hotflip.git
    %cd /kaggle/working/hotflip/
    !git pull origin main

refresh_repo()

/kaggle/working
Cloning into 'hotflip'...
remote: Enumerating objects: 473, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (249/249), done.
remote: Total 473 (delta 17), reused 255 (delta 12), pack-reused 212 (from 1)
Receiving objects: 100% (473/473), 45.91 MiB | 30.19 MiB/s, done.
Resolving deltas: 100% (111/111), done.
/kaggle/working/hotflip
From https://github.com/jefri021/hotflip
 * branch            main       -> FETCH_HEAD
Already up to date.


In [2]:
import torch
import json
import os
import logging
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn.functional as F

# def load_model(model_filepath: str, torch_dtype:torch.dtype=torch.float16):
#     """Load a model given a specific model_path.

#     Args:
#         model_filepath: str - Path to where the model is stored

#     Returns:
#         model, dict, str - Torch model + dictionary representation of the model + model class name
#     """

#     conf_filepath = os.path.join(model_filepath, 'reduced-config.json')
#     logging.info("Loading config file from: {}".format(conf_filepath))
#     with open(conf_filepath, 'r') as fh:
#         round_config = json.load(fh)

#     logging.info("Loading model from filepath: {}".format(model_filepath))
#     # https://huggingface.co/docs/transformers/installation#offline-mode
#     if round_config['use_lora']:
#         base_model_filepath = os.path.join(model_filepath, 'base-model')
#         logging.info("loading the base model (before LORA) from {}".format(base_model_filepath))
#         model = AutoModelForCausalLM.from_pretrained(base_model_filepath, device_map = "auto", trust_remote_code=True, torch_dtype=torch_dtype, local_files_only=True)
#         # model = AutoModelForCausalLM.from_pretrained(round_config['model_architecture'], trust_remote_code=True, attn_implementation="flash_attention_2", torch_dtype=torch_dtype)

#         fine_tuned_model_filepath = os.path.join(model_filepath, 'fine-tuned-model')
#         logging.info("loading the LORA adapter onto the base model from {}".format(fine_tuned_model_filepath))
#         model.load_adapter(fine_tuned_model_filepath)
#     else:
#         fine_tuned_model_filepath = os.path.join(model_filepath, 'fine-tuned-model')
#         logging.info("Loading full fine tune checkpoint into cpu from {}".format(fine_tuned_model_filepath))
#         model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_filepath, device_map = "auto", trust_remote_code=True, torch_dtype=torch_dtype, local_files_only=True)
#         # model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_filepath, trust_remote_code=True, attn_implementation="flash_attention_2", torch_dtype=torch_dtype)

#     model.eval()

#     tokenizer_filepath = os.path.join(model_filepath, 'tokenizer')
#     tokenizer = AutoTokenizer.from_pretrained(tokenizer_filepath)

#     return model, tokenizer


# def _two_gpu_max_memory(headroom_gb=2):
#     """
#     Reserve headroom so HF sharding MUST split across both 16GB T4s.
#     """
#     if not torch.cuda.is_available():
#         return None
#     n = torch.cuda.device_count()
#     cap = f"{16 - headroom_gb}GiB"  # e.g., "14GiB"
#     return {i: cap for i in range(n)}

# def _common_from_pretrained_kwargs():
#     """
#     Settings that reduce both CPU and GPU peak memory and use a lean attention impl.
#     """
#     kw = dict(
#         trust_remote_code=True,
#         local_files_only=True,
#         torch_dtype=torch.float16,     # T4 → FP16
#         low_cpu_mem_usage=True,        # streaming load
#         offload_state_dict=True,       # avoid CPU spikes
#         attn_implementation="sdpa",    # available by default on Kaggle
#     )
#     mm = _two_gpu_max_memory(headroom_gb=2)
#     if mm and torch.cuda.device_count() > 1:
#         kw["device_map"] = "auto"
#         kw["max_memory"] = mm
#         # Optional if host RAM is tight:
#         # kw["offload_folder"] = "/kaggle/working/offload"
#     else:
#         kw["device_map"] = {"": 0}
#     return kw

# def load_model_and_tokenizer(model_dir: str, merge_lora: bool = True):
#     """
#     Robust loader for full fine-tunes or LoRA adapters stored under `model_dir`.
#     Expects:
#       - reduced-config.json with {"use_lora": <bool>, ...}
#       - For LoRA: base-model/, fine-tuned-model/
#       - For full FT: fine-tuned-model/
#       - tokenizer/ with tokenizer files
#     Returns: (model, tokenizer)
#     """
#     conf_path = os.path.join(model_dir, "reduced-config.json")
#     logging.info(f"Loading config: {conf_path}")
#     with open(conf_path, "r") as fh:
#         cfg = json.load(fh)

#     kw = _common_from_pretrained_kwargs()

#     if cfg.get("use_lora", False):
#         base_dir = os.path.join(model_dir, "base-model")
#         lora_dir = os.path.join(model_dir, "fine-tuned-model")

#         logging.info(f"Loading base model: {base_dir}")
#         model = AutoModelForCausalLM.from_pretrained(base_dir, **kw)
#         logging.info(f"Attaching LoRA adapter: {lora_dir}")
#         # If PeftModel is missing, use .load_adapter if available
#         try:
#             model = PeftModel.from_pretrained(model, lora_dir, is_trainable=False)  # type: ignore
#         except Exception:
#             model.load_adapter(lora_dir)

#     else:
#         ft_dir = os.path.join(model_dir, "fine-tuned-model")
#         logging.info(f"Loading full fine-tuned model: {ft_dir}")
#         model = AutoModelForCausalLM.from_pretrained(ft_dir, **kw)

#     # Tokenizer hygiene
#     tok_dir = os.path.join(model_dir, "tokenizer")
#     tokenizer = AutoTokenizer.from_pretrained(tok_dir, use_fast=True, local_files_only=True)
#     if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
#         tokenizer.pad_token = tokenizer.eos_token
#     tokenizer.padding_side = "right"  # better for causal LMs with dynamic padding

#     # Runtime memory knobs for your gradient-based rollout
#     model.eval()
#     if hasattr(model.config, "use_cache"):
#         model.config.use_cache = False  # reduce KV/activation memory during your search

#     # Optional: quick sanity check of sharding
#     try:
#         print(getattr(model, "hf_device_map", "no device map"))
#     except Exception:
#         pass

#     return model, tokenizer

# model, tokenizer = load_model_and_tokenizer(
#     model_dir="/kaggle/input/trojai-rev2-00000001/id-00000001"
# )

from load_model import download_and_load
from load_model import load_model_and_tokenizer

mid = 4
fid = "1-K-HcT-3-00rxPpvQxZ75o2be3STchsv"

model, tokenizer = download_and_load(
    file_id=fid,
    output_filename=f"model{mid}.tar.gz",
    load_model_path=f"/kaggle/tmp/id-0000000{mid}")

2025-11-30 19:14:27.692434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764530067.713476     236 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764530067.719844     236 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Downloading...
From (original): https://drive.google.com/uc?id=1-K-HcT-3-00rxPpvQxZ75o2be3STchsv
From (redirected): https://drive.google.com/uc?id=1-K-HcT-3-00rxPpvQxZ75o2be3STchsv&confirm=t&uuid=e717a35e-92b4-41bb-9ec0-0e0361da1982
To: /kaggle/tmp/model4.tar.gz
100%|██████████| 10.6G/10.6G [00:57<00:00, 185MB/s] 


Download successful! File saved to: /kaggle/tmp/model4.tar.gz
File size: 10092.51 MB
Processing directory: /kaggle/tmp
Extracting: /kaggle/tmp/model4.tar.gz
Deleted compressed file: /kaggle/tmp/model4.tar.gz
Total .tar.gz files processed: 1


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 222.19 MiB is free. Process 10969 has 8.10 GiB memory in use. Process 16447 has 6.42 GiB memory in use. Of the allocated memory 6.29 GiB is allocated by PyTorch, and 30.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def get_emb_layer(model):
    model.eval()
    if hasattr(model.config, "use_cache"):
        model.config.use_cache = False
    return model.get_input_embeddings()

emb_layer = get_emb_layer(model)

In [ ]:
def project_suffix_to_tokens_and_diagnostics(
    suffix_z,
    emb_layer,
    tokenizer,
):
    """
    suffix_z: (Ls, E) - optimized continuous suffix embeddings
    emb_layer: model.get_input_embeddings()
    """
    with torch.no_grad():
        dev = emb_layer.weight.device
        E = emb_layer.weight        # (V, E)
        V, d = E.shape

        # Move suffix to same device
        z = suffix_z.to(dev)        # (Ls, E)

        # ---- Fix dtype mismatch: work in float32 for stability ----
        E_f = E.float()             # (V, E) fp32
        z_f = z.float()             # (Ls, E) fp32

        # Normalize for cosine similarity
        E_norm = F.normalize(E_f, dim=-1)        # (V, E)
        z_norm = F.normalize(z_f, dim=-1)        # (Ls, E)

        # Cosine similarity: (V, E) @ (E, Ls) -> (V, Ls)
        cos_sim = torch.matmul(E_norm, z_norm.T)  # (V, Ls)

        # For each suffix position, get best matching token
        best_token_ids = cos_sim.argmax(dim=0)    # (Ls,)

        # Diagnostics: L2 distances between z[i] and E[best_token_ids[i]]
        nearest_embs = E_f[best_token_ids]        # (Ls, E) fp32
        l2_dists = (z_f - nearest_embs).norm(dim=-1)  # (Ls,)

        print("L2 distance between optimized embeddings and nearest token embeddings:")
        print(f"  min:  {l2_dists.min().item():.6f}")
        print(f"  max:  {l2_dists.max().item():.6f}")
        print(f"  mean: {l2_dists.mean().item():.6f}")

        best_cos = cos_sim.max(dim=0).values     # (Ls,)
        print("Cosine similarity of optimized embeddings to nearest tokens:")
        print(f"  min:  {best_cos.min().item():.6f}")
        print(f"  max:  {best_cos.max().item():.6f}")
        print(f"  mean: {best_cos.mean().item():.6f}")

        suffix_token_ids = best_token_ids.cpu()
        suffix_tokens = tokenizer.convert_ids_to_tokens(suffix_token_ids.tolist())
        suffix_text = tokenizer.decode(
            suffix_token_ids.tolist(),
            skip_special_tokens=False
        )

        print("\nProjected discrete suffix token IDs:", suffix_token_ids.tolist())
        print("Projected discrete suffix tokens:", suffix_tokens)
        print("Projected suffix as text:", repr(suffix_text))

        return suffix_token_ids


In [ ]:
def read_suffix_pt(filepath: str) -> torch.Tensor:
    """
    Read suffix embeddings from a .pt file.
    """
    suffix_z = torch.load(filepath)
    return suffix_z

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch import amp

def entropy_loss(batch_logits):
    """
    batch_logits: (B, V) logits for the token of interest.
    Returns scalar mean entropy.
    """
    log_probs = F.log_softmax(batch_logits, dim=-1)
    probs = log_probs.exp()
    entropy = -(probs * log_probs).sum(dim=-1)  # (B,)
    return entropy.mean()

In [ ]:
def compute_loss_for_suffix(
    model,
    emb_layer,
    batch,
    suffix_z,           # (Ls, E) nn.Parameter
    n_tokens=10,
    amp_dtype=torch.float16,
    cos_reg_weight=0.1,
    E_norm_cpu=None,    # (V, E) on CPU, fp32
    chunk_size=1024,
    top_k=5,
    neg_weight=1.0,     # how strongly to push away from non-top-k
):
    """
    - For each example, build [prompt][suffix_z] in embedding space.
    - Pad all to same length -> [prompt][suffix][PAD].
    - Roll out n_tokens-1 tokens under inference_mode.
    - Final forward WITH grad gives entropy loss on last generated token.
    - Gradients flow into suffix_z only (prompts are detached).
    - PLUS: regularizer that pulls suffix_z toward real token embeddings via cosine similarity.
    """
    prompts = batch["input_ids"]   # list of 1D LongTensors (Li,)
    dev = emb_layer.weight.device
    suffix_z = suffix_z.to(dev)    # (Ls, E)

    B = len(prompts)
    Ls, E = suffix_z.shape

    base_embs = []   # each: (Li+Ls, E)
    base_lens = []   # each: scalar length Li+Ls

    # --- Build per-example [prompt][suffix] in embedding space ---
    for p_ids in prompts:
        p_ids_dev = p_ids.to(dev)
        p_emb = emb_layer(p_ids_dev).detach()   # (Li, E), prompts are constants
        base = torch.cat([p_emb, suffix_z], dim=0)  # (Li+Ls, E)
        base_embs.append(base)
        base_lens.append(base.size(0))

    # Pad to [prompt][suffix][PAD...] across the batch
    base = pad_sequence(base_embs, batch_first=True)   # (B, max_len, E)
    base_lens = torch.tensor(base_lens, device=dev)    # (B,)
    max_len = base.size(1)

    # Attention mask: 1 for real tokens, 0 for pad
    arange = torch.arange(max_len, device=dev).unsqueeze(0)  # (1, max_len)
    base_mask = (arange < base_lens.unsqueeze(1)).long()     # (B, max_len)

    # Now base has structure [prompt][suffix][PAD] per row (masked pads)

    def _one_step_logits(e, m):
        with amp.autocast("cuda", dtype=amp_dtype):
            out = model(
                inputs_embeds=e,
                attention_mask=m,
                use_cache=False,
                output_attentions=False,
                output_hidden_states=False,
                return_dict=True,
            )
        return out.logits[:, -1, :]  # (B, V)

    # ---------- Rollout under no grad (from detached base) ----------
    work_e = base.detach()  # rollout uses constants
    work_m = base_mask
    added_embs = []         # list of (B, E) constants
    
    T = max(0, n_tokens - 1)
    with torch.inference_mode():
        for _ in range(T):
            logits_t = _one_step_logits(work_e, work_m)
            probs_t  = torch.softmax(logits_t, dim=-1)
            next_ids = torch.argmax(probs_t, dim=-1)        # (B,)
    
            next_emb = emb_layer(next_ids.to(dev)).detach() # (B, E)
            added_embs.append(next_emb)
    
            work_e = torch.cat([work_e, next_emb.unsqueeze(1)], dim=1)
            work_m = torch.cat(
                [work_m, torch.ones((B, 1), dtype=work_m.dtype, device=dev)],
                dim=1,
            )
    
    # ---------- Final inputs: [prompt][suffix][PAD] + generated tokens ----------
    if len(added_embs) > 0:
        added = torch.stack(added_embs, dim=1)              # (B, T, E)
        final_emb = torch.cat([base, added], dim=1)         # (B, max_len+T, E)
        gen_mask = torch.ones((B, T), dtype=base_mask.dtype, device=dev)
        final_mask = torch.cat([base_mask, gen_mask], dim=1)
    else:
        final_emb = base
        final_mask = base_mask
    
    # ---------- Forward WITH grad for ALL n_tokens steps ----------
    with amp.autocast("cuda", dtype=amp_dtype):
        out = model(
            inputs_embeds=final_emb,
            attention_mask=final_mask,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
    
    logits_all = out.logits   # (B, L_total, V)
    B, L_total, V = logits_all.shape
    
    # base_lens: (B,) lengths of [prompt][suffix] BEFORE generated tokens
    # we want logits for:
    #   step 1: position base_len - 1  (first next token)
    #   step 2: position base_len      (second next)
    #   ...
    #   step n_tokens: position base_len - 1 + (n_tokens - 1) = base_len + T - 1
    # So indices: [base_len - 1, base_len, ..., base_len + T - 1], length = n_tokens
    
    all_step_logits = []
    
    for b in range(B):
        blen = base_lens[b].item()  # length of base for this example
    
        # safety: don't go past sequence length
        # we know we have exactly T generated tokens, so there are n_tokens positions:
        # indices from blen-1 to blen-1+T (inclusive)
        start_idx = blen - 1
        end_idx   = blen - 1 + T    # inclusive
        # this yields exactly n_tokens positions when T = n_tokens-1
    
        idxs = torch.arange(start_idx, end_idx + 1, device=dev)  # (n_tokens,)
        # gather logits for this example's steps: (n_tokens, V)
        step_logits_b = logits_all[b, idxs, :]                   # (n_tokens, V)
        all_step_logits.append(step_logits_b)
    
    # stack over batch: (B, n_tokens, V) -> (B*n_tokens, V)
    logits_for_loss = torch.cat(all_step_logits, dim=0)  # (B*n_tokens, V)

    # print(f"hey: {logits_for_loss.shape}")
    
    # mean entropy over all n_tokens steps for all examples
    ent = entropy_loss(logits_for_loss)

    
    dev = suffix_z.device
    Ls = suffix_z.size(0)
    V = E_norm_cpu.size(0)
    
    # normalized suffix embeddings on GPU, fp32, with grad
    z_norm = F.normalize(suffix_z.float(), dim=-1)  # (Ls, E)
    
    # running top-2 cosines across vocab, per suffix position
    top2_vals = None  # (2, Ls)
    
    for start in range(0, V, chunk_size):
        end = min(start + chunk_size, V)
        # chunk: (c, E) fp32 on GPU, no grad
        chunk = E_norm_cpu[start:end].to(dev, non_blocking=True)  # (c, E)
    
        # (c, E) @ (E, Ls) -> (c, Ls)
        chunk_sim = torch.matmul(chunk, z_norm.T)  # (c, Ls)
    
        # top-2 within this chunk: (2, Ls)  (if c < 2, topk handles it)
        chunk_top2, _ = chunk_sim.topk(
            k=min(2, chunk_sim.size(0)), dim=0
        )  # (k', Ls)
    
        if top2_vals is None:
            # if first chunk smaller than 2, pad with very low values
            if chunk_top2.size(0) < 2:
                pad_rows = 2 - chunk_top2.size(0)
                pad = torch.full(
                    (pad_rows, Ls),
                    -1e9,
                    device=dev,
                    dtype=chunk_top2.dtype,
                )
                top2_vals = torch.cat([chunk_top2, pad], dim=0)  # (2, Ls)
            else:
                top2_vals = chunk_top2  # (2, Ls)
        else:
            # combine global and chunk top2, then keep best 2 across all
            combined = torch.cat([top2_vals, chunk_top2], dim=0)  # (2 + k', Ls)
            top2_vals, _ = combined.topk(k=2, dim=0)              # (2, Ls)
    
        # free small temps
        del chunk, chunk_sim, chunk_top2
    
    # Now top2_vals[0] = global best cosine; top2_vals[1] = global second-best
    top1 = top2_vals[0]   # (Ls,)
    top2 = top2_vals[1]   # (Ls,)
    
    # margin per position: encourage top1 >> top2
    margin_per_pos = top1 - top2  # (Ls,)
    mean_margin = margin_per_pos.mean()            # scalar
    
    # We want to MAXIMIZE mean_margin → in minimization, use negative margin
    margin_reg = -mean_margin
    
    cos_reg = margin_reg  # you can scale it directly with cos_reg_weight below
    
    total_loss = ent + cos_reg_weight * cos_reg
    
    return total_loss

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset

def load_prompts_unpadded(tokenizer, args):
    """
    Returns DataLoader where each batch is:
      {
        "input_ids": list of 1D LongTensors (prompts, no padding),
        "prompt_lens": LongTensor (B,)
      }
    """
    ds = load_dataset("tatsu-lab/alpaca", split="train", cache_dir=args["data_dir"])

    # Subsample for speed
    if "sample_size" in args and args["sample_size"] is not None and args["sample_size"] < len(ds):
        ds = ds.shuffle(seed=42).select(range(args["sample_size"]))

    def collate(batch):
        texts = [ex["instruction"] for ex in batch]
        enc = tokenizer(
            texts,
            padding=False,
            truncation=True,
            max_length=args["max_length"],
        )
        prompts = [torch.tensor(ids, dtype=torch.long) for ids in enc["input_ids"]]
        prompt_lens = [len(p) for p in prompts]

        return {
            "input_ids": prompts,  # list of (Li,)
            "prompt_lens": torch.tensor(prompt_lens, dtype=torch.long),
        }

    num_workers = max(2, os.cpu_count() // 2)
    return DataLoader(
        ds,
        batch_size=args["batch_size"],
        shuffle=True,
        pin_memory=True,
        num_workers=num_workers,
        persistent_workers=True,
        collate_fn=collate,
    )

dataloader_args = {
    "data_dir": "/kaggle/working/datasets",
    "batch_size": 1,
    "max_length": 128,
    "sample_size": 128,
}

dataloader = load_prompts_unpadded(tokenizer, dataloader_args)


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
def generate_with_suffix(
    model,
    tokenizer,
    prompt_ids: torch.Tensor,        # (L_prompt,)
    suffix_token_ids: torch.Tensor,  # (Ls,)
    max_new_tokens: int = 50,
):
    """
    Concatenate [prompt][suffix] and let the model generate continuation.
    """
    device = next(model.parameters()).device

    full_input_ids = torch.cat(
        [prompt_ids, suffix_token_ids.to(prompt_ids.device)],
        dim=0
    ).unsqueeze(0).to(device)  # (1, L_total)

    attn_mask = torch.ones_like(full_input_ids, dtype=torch.long, device=device)

    model.eval()
    with torch.no_grad():
        generated = model.generate(
            input_ids=full_input_ids,
            attention_mask=attn_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )

    return tokenizer.decode(generated[0], skip_special_tokens=True)

In [ ]:
import torch.nn.functional as F

def log_suffix_token_probs(
    model,
    tokenizer,
    prompt_ids: torch.Tensor,        # (L_prompt,)
    suffix_token_ids: torch.Tensor,  # (Ls,)
):
    """
    For each position k in the suffix, compute P(suffix[k] | prompt + suffix[:k])
    and print it.
    """
    device = next(model.parameters()).device
    model.eval()

    prompt_ids = prompt_ids.to(device)
    suffix_token_ids = suffix_token_ids.to(device)

    with torch.no_grad():
        for k in range(suffix_token_ids.size(0)):
            ctx_suffix = suffix_token_ids[:k]      # (k,)
            ctx = torch.cat([prompt_ids, ctx_suffix], dim=0)  # (L_prompt + k,)

            inp = ctx.unsqueeze(0)  # (1, L_ctx)
            msk = torch.ones_like(inp, dtype=torch.long, device=device)

            out = model(
                input_ids=inp,
                attention_mask=msk,
                use_cache=False,
                return_dict=True,
            )
            logits_next = out.logits[:, -1, :]  # (1, V)
            probs_next = F.softmax(logits_next, dim=-1)  # (1, V)

            tok_id = suffix_token_ids[k]
            prob = probs_next[0, tok_id].item()
            tok_str = tokenizer.decode([tok_id])

            print(f"  pos {k:2d}, token {tok_id:5d} ({repr(tok_str)}), prob: {prob:.6f}")

In [ ]:
from itertools import islice

device = next(model.parameters()).device
emb_layer = model.get_input_embeddings()

# Get one batch iterator for prompts
data_iter = iter(dataloader)
batch = next(data_iter)
prompts = batch["input_ids"]  # list of 1D tensors
prompt_ids = prompts[0]       # pick first prompt
print("Using this prompt for qualitative checks:")
print("  ", tokenizer.decode(prompt_ids, skip_special_tokens=True))
print()

with torch.no_grad():
    E_cpu = model.get_input_embeddings().weight.detach().cpu().float()  # (V, E)
    E_norm_cpu = F.normalize(E_cpu, dim=-1)  # (V, E), fp32 on CPU

for i in range(4):      # rounds
    for j in range(10): # epochs/checkpoints per round
        path = f"/kaggle/working/hotflip/rounds/clean/suffix_r{i}_e{j}.pt"

        try:
            suffix_z = read_suffix_pt(path)  # (Ls, E) on CPU
        except FileNotFoundError:
            print(f"[WARN] Missing file: {path}")
            continue

        print(f"\n=== round {i}, epoch {j} ===")
        print(f"Read {path} successfully. shape={tuple(suffix_z.shape)}")

        # Project to tokens + diagnostics
        suffix_token_ids = project_suffix_to_tokens_and_diagnostics(
            suffix_z,
            emb_layer,
            tokenizer,
        )  # LongTensor (Ls,)

        # Compute loss BEFORE projection (continuous suffix)
        batch_eos = {"input_ids": [torch.tensor([tokenizer.eos_token_id], dtype=torch.long)]}
        loss_before = compute_loss_for_suffix(
            model,
            emb_layer,
            batch_eos,
            suffix_z.to(device),
            n_tokens=10,
            amp_dtype=torch.float16,
            cos_reg_weight=1.0,
            E_norm_cpu=E_norm_cpu,
        )
        print(f"suffix loss (before projection): {loss_before.item():.6f}")

        # Build suffix_z AFTER projection: embeddings of discrete tokens
        suffix_z_proj = emb_layer(suffix_token_ids.to(device))  # (Ls, E)

        loss_after = compute_loss_for_suffix(
            model,
            emb_layer,
            batch_eos,
            suffix_z_proj,
            n_tokens=10,
            amp_dtype=torch.float16,
            cos_reg_weight=1.0,
            E_norm_cpu=E_norm_cpu,
        )
        print(f"suffix loss (after  projection): {loss_after.item():.6f}")

        # ---- Qualitative check: generate from a real prompt + suffix ----
        print("\nGenerated text with suffix (projected):")
        gen_text = generate_with_suffix(
            model,
            tokenizer,
            prompt_ids,
            suffix_token_ids,
            max_new_tokens=64,
        )
        print(gen_text)
        print()

        # ---- Per-suffix-token probabilities given the prompt ----
        print("Per-suffix-token next-token probabilities:")
        log_suffix_token_probs(
            model,
            tokenizer,
            prompt_ids,
            suffix_token_ids,
        )

        print("####################")


Using this prompt for qualitative checks:
   List three sites of historical importance in London.


=== round 0, epoch 0 ===
Read /kaggle/working/hotflip/rounds/suffix_r0_e0.pt successfully. shape=(10, 4096)
L2 distance between optimized embeddings and nearest token embeddings:
  min:  3.787844
  max:  4.888620
  mean: 4.140403
Cosine similarity of optimized embeddings to nearest tokens:
  min:  0.130694
  max:  0.297501
  mean: 0.202913

Projected discrete suffix token IDs: [20609, 1642, 7713, 10056, 10805, 29169, 3798, 1858, 7236, 25775]
Projected discrete suffix tokens: ['▁Portály', '".', 'Pol', '▁Consider', '▁causing', '²).', 'oreferrer', '▁Pl', '▁Fort', '▁Ari']
Projected suffix as text: 'Portály".Pol Consider causing²).oreferrer Pl Fort Ari'
suffix loss (before projection): 4.947422


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


suffix loss (after  projection): 5.449004

Generated text with suffix (projected):
List three sites of historical importance in London. Portály".Pol Consider causing²).oreferrer Pl Fort Ari. 1. The Tower of London 2. The Houses of Parliament 3. Buckingham Palace The Tower of London is a castle built by William the Conqueror in 1066. It was built to protect London from invaders. It was also used to keep prisoners. The Houses

Per-suffix-token next-token probabilities:
  pos  0, token 20609 ('Portály'), prob: 0.000000
  pos  1, token  1642 ('".'), prob: 0.000004
  pos  2, token  7713 ('Pol'), prob: 0.000015
  pos  3, token 10056 ('Consider'), prob: 0.000000
  pos  4, token 10805 ('causing'), prob: 0.000001
  pos  5, token 29169 ('²).'), prob: 0.000000
  pos  6, token  3798 ('oreferrer'), prob: 0.000000
  pos  7, token  1858 ('Pl'), prob: 0.000015
  pos  8, token  7236 ('Fort'), prob: 0.000002
  pos  9, token 25775 ('Ari'), prob: 0.000004
####################

=== round 0, epoch 1 ===
Rea

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


suffix loss (after  projection): 5.504870

Generated text with suffix (projected):
List three sites of historical importance in London. Portály".Mark Consider causing²).oreferrerMag Fort Ariane. 1. The Tower of London. 2. The Houses of Parliament. 3. The British Museum. 4. The British Library. 5. The National Gallery. 6. The National Portrait Gallery. 7. The Royal Opera House. 8. The Royal Academy of

Per-suffix-token next-token probabilities:
  pos  0, token 20609 ('Portály'), prob: 0.000000
  pos  1, token  1642 ('".'), prob: 0.000004
  pos  2, token  9802 ('Mark'), prob: 0.000019
  pos  3, token 10056 ('Consider'), prob: 0.000001
  pos  4, token 10805 ('causing'), prob: 0.000006
  pos  5, token 29169 ('²).'), prob: 0.000000
  pos  6, token  3798 ('oreferrer'), prob: 0.000000
  pos  7, token 19095 ('Mag'), prob: 0.000001
  pos  8, token  7236 ('Fort'), prob: 0.000017
  pos  9, token 25775 ('Ari'), prob: 0.000005
####################

=== round 0, epoch 2 ===
Read /kaggle/working/hotf

...

suffix loss (after  projection): 4.915522

Generated text with suffix (projected):
List three sites of historical importance in London. Sp More。 meter Screen P Mediabestanden M TMa.
































































Per-suffix-token next-token probabilities:
  pos  0, token  1706 ('Sp'), prob: 0.000304
  pos  1, token  5853 ('More'), prob: 0.000000
  pos  2, token 30267 ('。'), prob: 0.000000
  pos  3, token 11134 ('meter'), prob: 0.000000
  pos  4, token 22666 ('Screen'), prob: 0.000000
  pos  5, token   349 ('P'), prob: 0.000282
  pos  6, token 28574 ('Mediabestanden'), prob: 0.000000
  pos  7, token   341 ('M'), prob: 0.001537
  pos  8, token   323 ('T'), prob: 0.009132
  pos  9, token 21870 ('Ma'), prob: 0.000004
####################

=== round 3, epoch 8 ===
Read /kaggle/working/hotflip/rounds/suffix_r3_e8.pt successfully. shape=(10, 4096)
L2 distance between optimized embeddings and nearest token embeddings:
  min:  2.962456
  max:  7.777992
  mean: 4

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


suffix loss (after  projection): 4.915522

Generated text with suffix (projected):
List three sites of historical importance in London. Sp More。 meter Screen P Mediabestanden M TMa.
































































Per-suffix-token next-token probabilities:
  pos  0, token  1706 ('Sp'), prob: 0.000304
  pos  1, token  5853 ('More'), prob: 0.000000
  pos  2, token 30267 ('。'), prob: 0.000000
  pos  3, token 11134 ('meter'), prob: 0.000000
  pos  4, token 22666 ('Screen'), prob: 0.000000
  pos  5, token   349 ('P'), prob: 0.000282
  pos  6, token 28574 ('Mediabestanden'), prob: 0.000000
  pos  7, token   341 ('M'), prob: 0.001537
  pos  8, token   323 ('T'), prob: 0.009132
  pos  9, token 21870 ('Ma'), prob: 0.000004
####################

=== round 3, epoch 9 ===
Read /kaggle/working/hotflip/rounds/suffix_r3_e9.pt successfully. shape=(10, 4096)
L2 distance between optimized embeddings and nearest token embeddings:
  min:  2.738678
  max:  7.797936
  mean: 4

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


suffix loss (after  projection): 4.915522

Generated text with suffix (projected):
List three sites of historical importance in London. Sp More。 meter Screen P Mediabestanden M TMa.
































































Per-suffix-token next-token probabilities:
  pos  0, token  1706 ('Sp'), prob: 0.000304
  pos  1, token  5853 ('More'), prob: 0.000000
  pos  2, token 30267 ('。'), prob: 0.000000
  pos  3, token 11134 ('meter'), prob: 0.000000
  pos  4, token 22666 ('Screen'), prob: 0.000000
  pos  5, token   349 ('P'), prob: 0.000282
  pos  6, token 28574 ('Mediabestanden'), prob: 0.000000
  pos  7, token   341 ('M'), prob: 0.001537
  pos  8, token   323 ('T'), prob: 0.009132
  pos  9, token 21870 ('Ma'), prob: 0.000004
####################


In [ ]:
!ls /kaggle/working/hotflip/rounds

suffix_r0_e0.pt  suffix_r1_e0.pt  suffix_r2_e0.pt  suffix_r3_e0.pt
suffix_r0_e1.pt  suffix_r1_e1.pt  suffix_r2_e1.pt  suffix_r3_e1.pt
suffix_r0_e2.pt  suffix_r1_e2.pt  suffix_r2_e2.pt  suffix_r3_e2.pt
suffix_r0_e3.pt  suffix_r1_e3.pt  suffix_r2_e3.pt  suffix_r3_e3.pt
suffix_r0_e4.pt  suffix_r1_e4.pt  suffix_r2_e4.pt  suffix_r3_e4.pt
suffix_r0_e5.pt  suffix_r1_e5.pt  suffix_r2_e5.pt  suffix_r3_e5.pt
suffix_r0_e6.pt  suffix_r1_e6.pt  suffix_r2_e6.pt  suffix_r3_e6.pt
suffix_r0_e7.pt  suffix_r1_e7.pt  suffix_r2_e7.pt  suffix_r3_e7.pt
suffix_r0_e8.pt  suffix_r1_e8.pt  suffix_r2_e8.pt  suffix_r3_e8.pt
suffix_r0_e9.pt  suffix_r1_e9.pt  suffix_r2_e9.pt  suffix_r3_e9.pt


In [ ]:
refresh_repo()

/kaggle/working
Cloning into 'hotflip'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 341 (delta 11), reused 127 (delta 10), pack-reused 212 (from 1)
Receiving objects: 100% (341/341), 28.44 MiB | 21.22 MiB/s, done.
Resolving deltas: 100% (105/105), done.
/kaggle/working/hotflip
From https://github.com/jefri021/hotflip
 * branch            main       -> FETCH_HEAD
Already up to date.
